In [1]:
from IPython.core.debugger import set_trace
from torchtext.datasets import WikiText2, IMDB
import spacy
import re
import html
from torchtext import data, datasets
from spacy.symbols import ORTH
import torch
import torch.nn as nn
import torch.nn.functional as V
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [2]:
spacy_en  = spacy.load('en')

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [3]:
def tokenizer(x):
    return [tok.text for tok in spacy_en.tokenizer(x)]

In [4]:
TEXT = data.Field(lower=True, tokenize = tokenizer)


In [5]:
train, valid, test = WikiText2.splits(TEXT)

In [6]:
TEXT.build_vocab(train, vectors = "fasttext.en.300d")

In [7]:
train_iter, valid_iter, test_iter = data.BPTTIterator.splits(
    (train, valid, test),
    batch_size=16,
    bptt_len=30, # this is where we specify the sequence length
    device = "cuda",
    repeat=False)

In [23]:
class LanguageModel(nn.Module):
    
    def __init__(self, ntoken, ninp,
                 nhid, nlayers, bsz,
                 dropout=0.5):
        super(LanguageModel, self).__init__()
        self.nhid, self.nlayers, self.bsz = nhid, nlayers, bsz
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout,bidirectional=True)
        self.decoder = nn.Linear(nhid*2,ntoken)

        self.init_weights()
        self.hidden = self.init_hidden(bsz)
        
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)
 
    def forward(self, input):
        emb = self.drop(self.encoder(input))
        output, self.hidden = self.rnn(emb, self.hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1))
 
    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        return (torch.tensor(weight.new(self.nlayers*2, bsz, self.nhid).zero_().cuda()),
                torch.tensor(weight.new(self.nlayers*2, bsz, self.nhid).zero_()).cuda())
  
    def reset_history(self):
        self.hidden = tuple(torch.tensor(h.data) for h in self.hidden)


In [24]:
weight_matrix = TEXT.vocab.vectors
model = LanguageModel(weight_matrix.size(0),
weight_matrix.size(1), 200, 3, 16, dropout = 0.6)
model.encoder.weight.data.copy_(weight_matrix)
model.cuda()

LanguageModel(
  (drop): Dropout(p=0.6)
  (encoder): Embedding(28870, 300)
  (rnn): LSTM(300, 200, num_layers=3, dropout=0.6, bidirectional=True)
  (decoder): Linear(in_features=400, out_features=28870, bias=True)
)

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3, betas=(0.7,0.99))
n_tokens = weight_matrix.size(0)

In [26]:
from tqdm import tqdm_notebook as tqdm

def train_epoch(epoch):
    epoch_loss = 0
    for batch in tqdm(train_iter):
        model.reset_history()
    
        optimizer.zero_grad()
        
        text, targets = batch.text, batch.target
        prediction = model(text)
        loss = criterion(prediction.view(-1, n_tokens), targets.view(-1))
        loss.backward()
        
        optimizer.step()
        #set_trace()
        batch_loss = loss.item() * prediction.size(0) * prediction.size(1)
        
        batch_loss /= len(train.examples[0].text)
        
        epoch_loss += batch_loss
    
    val_loss = 0
        
    for index, batch in enumerate(tqdm(valid_iter)):
        model.reset_history()
        text, targets = batch.text, batch.target
        prediction = model(text)
        loss = criterion(prediction.view(-1, n_tokens), targets.view(-1))
        batch_loss = loss.item() * prediction.size(0) * prediction.size(1)
        batch_loss /= len(valid.examples[0].text) 
        
        val_loss += batch_loss
        
    print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(epoch, epoch_loss, val_loss))
    return val_loss
        

In [27]:
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=1, verbose = True)
for i in range(30):
    val_losses = []
    val_loss = train_epoch(i)
    val_losses.append(val_loss)
    scheduler.step(val_loss)
    if val_loss == min(val_losses):
        save_path = 'lstm_min_loss.pt'
        torch.save(model.state_dict(), save_path)

KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.loadd('lstm_min_loss.pt'))

In [30]:
batch_size = 16

In [42]:
class ClassifierModel(nn.Module):
    
    def __init__(self, ntoken, ninp,
                 nhid, nlayers, bsz, noutputs,
                 dropout=0.8):
        super(ClassifierModel, self).__init__()
        self.nhid, self.nlayers, self.bsz = nhid, nlayers, bsz
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout, bidirectional=True)
        self.linear = nn.Linear(nhid*2,noutputs)

    def forward(self, input):
        bsz = input.size()[1]
        if bsz != self.bsz:
            self.bsz = bsz
        emb = self.drop(self.encoder(input))
        self.hidden = (torch.tensor(emb.data.new(*(self.nlayers*2, self.bsz, self.nhid)).zero_()),
                       torch.tensor(emb.data.new(*(self.nlayers*2, self.bsz, self.nhid)).zero_()))
        output, _ = self.rnn(emb, self.hidden)
        output = self.drop(torch.cat((output[-2,:,:], output[-1,:,:]), dim=1))
        return self.linear(output)


In [43]:
model2 = ClassifierModel(weight_matrix.size(0),
weight_matrix.size(1), 200, 3, batch_size,2,dropout = 0.8)
model2.cuda()

ClassifierModel(
  (drop): Dropout(p=0.8)
  (encoder): Embedding(28870, 300)
  (rnn): LSTM(300, 200, num_layers=3, dropout=0.8, bidirectional=True)
  (linear): Linear(in_features=400, out_features=2, bias=True)
)

In [38]:
model1_state = model.state_dict()
model2_state = model2.state_dict()
pretrained_state = { k:v for k,v in model1_state.items() if k in model2_state and v.size() == model2_state[k].size() }
model2_state.update(pretrained_state)
model2.load_state_dict(model2_state)

In [34]:
LABEL = data.LabelField(tokenize='spacy')
train2, test2 = datasets.IMDB.splits(TEXT,LABEL)

In [39]:
train_iter2, test_iter2 = data.BucketIterator.splits((train2, test2), device='cuda', batch_size=batch_size,shuffle=True)
LABEL.build_vocab(train2)

In [44]:
optimizer = optim.Adam(model2.parameters(),lr=1e-3)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, verbose = True)
loss_function = nn.CrossEntropyLoss()
loss_function = loss_function.cuda()
def fit(epoch,model2,data_loader,phase='training'):
    if phase == 'training':
        model2.train()
    if phase == 'validation':
        model2.eval()
    running_loss = 0.0
    running_correct = 0
    run_total = 0
    for batch_idx , batch in enumerate(data_loader):
        text , target = batch.text , batch.label
        
        if phase == 'training':
            optimizer.zero_grad()
        output = model2(text)
        output = output.squeeze(1)
        loss = loss_function(output,target)
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += (preds.squeeze() == target).float().sum()
        run_total += len(target)
        running_loss += loss.detach()
        if phase == 'training':            
            loss.backward()
            optimizer.step()
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * float(running_correct)/float(run_total)
    print('Epoch: {}, Loss: {:.4f}, Accuracy: {:.4f}'.format(epoch, loss, accuracy))
    return loss,accuracy
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

In [ ]:
print("train_losses")
print(train_losses)
print("train_accuracy")
print(train_accuracy)
print("val_losses")
print(val_losses)
print("val_accuracy")
print(val_accuracy)

## Rerun the Experiment without pre-training

In [ ]:
%debug

> /home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/functional.py(1024)linear()
   1022     if input.dim() == 2 and bias is not None:
   1023         # fused op is marginally faster
-> 1024         return torch.addmm(bias, input, weight.t())
   1025 
   1026     output = input.matmul(weight.t())

ipdb> u
> /home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/linear.py(55)forward()
     53 
     54     def forward(self, input):
---> 55         return F.linear(input, self.weight, self.bias)
     56 
     57     def extra_repr(self):

ipdb> u
> /home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/module.py(477)__call__()
    475             result = self._slow_forward(*input, **kwargs)
    476         else:
--> 477             result = self.forward(*input, **kwargs)
    478         for hook in self._forward_hooks.values():
    479             hook_result = hook(self, input, result)

ipdb> u
> <ipython-

In [ ]:
model3 = ClassifierModel(weight_matrix.size(0),
weight_matrix.size(1), 200, 3, batch_size,2,dropout = 0.8)
model3.cuda()
optimizer = optim.Adam(model3.parameters(),lr=1e-3)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, verbose = True)
loss_function = nn.CrossEntropyLoss()
loss_function = loss_function.cuda()
train_losses2 , train_accuracy2 = [],[]
val_losses2 , val_accuracy2 = [],[] 

for epoch in tqdm(range(30)):
    epoch_loss, epoch_accuracy = fit(epoch,model3,train_iter2,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model3,test_iter2,phase='validation')
    scheduler.step(val_epoch_loss)
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)